# Introducción a SQL
***
<div class="panel panel-danger">
    <div class='panel-heading'>
    <h4>Antes de empezar</h4>
    </div>
    <div class='panel-body'>
    <p>Para poder ejecutar queries a la base de datos, necesitamos tener instalado antes el paquete [ipython-sql](https://github.com/catherinedevlin/ipython-sql).
    
    <p>Este paquete nos permite escribir las queries sql en las celdas del notebook y visualizar el resultado como tablas html renderizadas
    <p>Ejecuta `!pip install ipython-sql` si es la primera vez que usas este notebook
    </div>
</div>

In [ ]:
!pip install ipython-sql

Una vez instalado, podemos invocar el entorno `SQL` usando `%load_ext sql`.
Esto nos permitira usar el magic de ipython `%%sql` al principo de **cada** celda y ejecutar comandos sql directamente

In [39]:
# Esto carga el paquete instalado y nos conecta con la base de datos contenida en el fichero "bdd_notas.sqlite"
%load_ext sql
%sql sqlite:///bdd_notas.sqlite

%sql SELECT name FROM sqlite_master WHERE type='table'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
Done.


name
estudiantes
asignaturas
notas


<div class="panel panel-success">
    <div class='panel-heading'>
    <h4>Empecemos</h4>
    </div>
</div>


Los objetivos de este notebook son:

1. Entender la sintaxis SQL básica
2. Entender la diferencia entre "inner", “left”, “right” y “full” joins
3. Entender groupings y como se conectan a las funciones de agregación
4. Como ejecutar queries SQL y guardar los resultados en pandas.DataFrame()

# 1. SQL: sintaxis básica

### Nombres clave:

- Relational database
- Table
- Row
- Column
- (Primary key)
- (Foreign key)

### Verbos clave:

* `CREATE TABLE`, `DROP TABLE`: Crear o eliminar tablas
* <span style="color:green">SELECT:</span>: Seleccionar las columnas deseadas de la tabla
* `INSERT` / `UPDATE` / `DELETE`: para crear, modificar, o borrar filas de una tabla

### Modificadores clave:
* <span style="color:green">WHERE</span>: imponer una condición/filtro sobre los valores de las columnas
* <span style="color:green">ORDER BY</span>: ordenar
* <span style="color:green">GROUP BY</span>: agregar los valores por un campo
* <span style="color:green">JOIN</span>: combinar tablas en una query
* <span style="color:green">HAVING</span>: impone una condición/filtro sobre valores agregados por el `GROUP BY`
* <span style="color:green">LIMIT</span>: el límite de valores que queremos sacar

Las queries en SQL se ejecutan contra una base de datos combinando los diferentes verbos y los modificadores clave.

Al principio de éste notebook nos hemos conectado a una base de datos de juguete para practicar como construir queries. Existen las siguientes columnas:

Columnas|
:--------:|
estudiantes  | 
asignaturas | 
notas |

**Nota final:**

SQL no es sensible a las mayúsculas o minúsculas y a los cambios de linea y espacios.

Por convenio, se usan mayusculas para las palabras clave y minusculas para el nombre de los campos o columnas.

También es aconsejable usar una nueva linea para cada comando.

### 1. Seleccionar valores de columnas

```SQL
SELECT column_name1,
       column_name2
FROM table_name
[LIMIT 100];
```
Si en SELECT indicamos un \*, apareceran todas las columnas

LIMIT va entre entre braquets para indicar un campo opcional

In [2]:
%%sql

SELECT *
FROM estudiantes
LIMIT 3

Done.


student_id,name
101,Juan
102,Maria
103,Pedro


### Ejercicio 1: Devuelve todos los campos de la tabla `notas`. Limita la salida a 5 elementos

### Ejercicio 2: Devuelve todos los campos de la tabla `asignaturas`


En SQL también podemos asignar nombres a columnas. Por ejemplo usando la palabre clave `as`.

Dos de las expresiones básicas en SQL que podemos aplicar a los nombres de columnas son:

`COUNT` : Nos devuelve un conteo de los elementos especificados por la query

`DISTINCT`: elimina los duplicados

Por ejemplo, si quisieramos saber cuantas filas tiene una tabla de nuestra base de datos podriamos ejecutar 

```sql
SELECT COUNT(*) as num_rows
FROM my_table
```

Y si quisieramos encontrar el número de elementos distintos de una columna, podríamos encadenar las expresiones:

```sql
SELECT COUNT(DISTINCT col_name) as unique_rows
FROM my_table
```

### Ejercicio: Encuentra el numero de filas total de cada una de las tablas de la base de datos y asignales un nombre

### Ejercicio: Cuantas notas distintas hay en la columna `nota` de la tabla `notas`?

## Filtrando y Ordenando los datos
---
Para poder filtrar según una condición que queramos imponer, por ejemplo las notas que sean menores que 5, debemos usar el modificador **`WHERE`** seguida de la condición que queremos poner sobre alguna de las columnas.

### Ejercicio: Cuanta gente ha suspendido?

Además de filtrar, podemos definir el orden con el que queremos que nos devuelva los datos, ascendente o descendente. Combinado con el LIMIT, esto nos permitirá responder a preguntas tipo cual es el Top 5 o cual es la segunda mejor nota.

Por ejemplo, la query:

```sql
SELECT year, nota
from notas
WHERE year = '2015'
ORDER BY nota DESC
LIMIT 1
```
Nos devolverá la mejor nota del año 2015.

### Ejercicio: Encuentra la peor nota del año 2017

A la hora de trabajar con campos no numéricos, strings, podemos filtrar mediante LIKE. Con el podremos comparar cadenas y subcadenas de caracteres. Para poder especificar un inicio o un final abierto, usaremos '%' en el lugar de la cadena que queramos. Por ejemplo

```sql
WHERE name LIKE('J%')
```

Nos devolvería todos los elementos de la columna `name` que empiecen por J, mientras que:

```sql
WHERE name LIKE('%S')
```
Nos devolvería todos los nombres que acaban por S.

Finalmente,

```sql
WHERE name LIKE('%er%')
```
Nos devolvería aquellos nombres que contuvieran la subcadena `er` en cualquier lugar de la cadena

### Ejercicio: Encuentra todos los estudiantes que contienen una `a` en el nombre

## Condicionales
SQL permite usar logica condicional sencilla. Es el equivalente a una expresión condicional `if-else`.

Se utiliza después del SELECT e iniciandolo con la palabra clave `CASE`. Después de abrir el condicional, deberemos explicitar las diferentes opciones usando la notación `WHEN` this `THEN` that.

Cuando queramos agrupar las opciones restantes, podemos hacerlo con `ELSE` something. 

Para cerrar el condicional, deberemos explicitarlo con `END` seguido de un alias para la columna condicional que acabamos de crear.

Veamos un ejemplo sencillo en el que crearemos una columna para la gente que haya aprobado/suspendido.

```sql
SELECT asign_id,
       CASE
       WHEN nota < 5 THEN 'suspenso'
       ELSE "aprobado"
       END as "estado_asignatura"
FROM notas
```

#### Ejercicio: Copia y pega la expresión anterior y verifica que funciona. Modifica la función de tal forma que la gente que haya sacado más de un 8 obtenga el valor `matricula`

#### Ejercicio: Añade un filtro a la expresión anterior de forma que solo nos de las notas de la asignatura de mates (id: 201) y ordenadas por el año de menor a mayor

## Agregaciones: `GROUP BY`
   
Finalmente, SQL también nos permite agrupar columnas usando distintas formas de agregación.

Con tal de poder usarlas, debemos espedificar en el `GROUP BY col1, col2` las columnas de la tabla que mencionamos en el `SELECT` que queremos agrupar. Esto implica dar una agrupación a alguna de las columnas restantes.

Las más frecuentes son:
* COUNT()
* SUM()
* AVG()
* MIN()
* MAX()
   
Por ejemplo,

```SQL 
SELECT asign_id,
       AVG(nota) as nota_media
FROM notas
GROUP BY asign_id;
```
Nos devolvería la nota media de cada una de las asignaturas.

### Ejercicio: Cómo han evolucionado la nota máxima, mínima y media de cada asignatura a lo largo de los últimos años?

## Filtrando datos agregados

Así como podemos imponer condiciones de filtrado sobre columnas de nuestro `SELECT`, podemos especificar condiciones de filtrado para nuestros campos agregados.

Por ejemplo, si quisieramos listar aquellas asignaturas que tienen una media superior a 6, deberíamos añadir

```sql
HAVING AVG(nota) > 6
```

A nuestra query, que quedaría:

```sql
SELECT asign_id, AVG(nota) as nota_media
from notas
GROUP BY asign_id
HAVING AVG(nota) > 6
```

##### Ejercicio: Cual es el codigo de las asignaturas que tuvieron una nota media de aprobado en el año 2016

# RESUMEN

En resumen, la query básica para leer datos de **Una** tabla de nuestra base de datos sería:

```SQL
SELECT expression1, expression2, ...
FROM table_name
[WHERE condition]
[GROUP BY columns]
[HAVING condition]
[ORDER BY columns]
[LIMIT number];
```

# Combinando tablas: joins

Hasta ahora, hemos visto como hacer consultas básicas a tablas de forma independiente, no está mal, pero es bastante aburrido y laborioso.

Imagina que queremos encontrar quién es el/la mejor estudiante. Podríamos obtener esta información consultando la tabla notas, pero, cómo saber el nombre? En ese caso, deberiamos consultar en la base de datos la tabla que contiene el id del estudiante y filtrar por aquel id que nos haya devuelto la query anterior.

Esto puede hacerse de forma programática usando los `joins` de sql. Esta opción nos permite combinar información de distintas tablas y hacer consultas sobre este nuevo conjunto. Por supuesto, podríamos hacer esto y guardar los resultados en una nueva tabla, pero la habilidad de poder combinar tablas "al vuelo" nos permite un ahorro de memoria y redundancia de datos.

La imagen siguiente muestra un resumen visual de los distintos joins que existen y la sintaxis de como obtenerlos.


<img src='img/sql-joins.jpg'>

La diferencia básica entre los distintos `joins` es como gestiona la ausencia de datos de una tabla en la otra.

Por ejemplo si tenemos las tablas siguientes con los colores favoritos de nuestros amigos :

 sweater_color | price 
:-------------:|:-----:
      rojo      | 59.99 
    amarillo     | 49.99 
     azul      | 79.99 
    verde     | 89.99 

   name   |  city   | favorite_color 
:--------:|:-------:|:--------------:
  Juan    | Inca |     amarillo     
 Maria    | Palma |     rojo
 Pedro | Campanet | marron

Un inner join nos devolveria la intersección de las dos, es decir:


   name   |  city   | favorite_color| price
:--------:|:-------:|:--------------:|----
  Juan    | Inca |     amarillo | 49.99 
 Maria    | Palma |     rojo | 59.99
 
 Hay que tener en cuenta que los joins son direccionales, es decir que la primera tabla que se especifica es aquella que tiene preferencia.
 
Por ejemplo, si hacemos un left join de la tabla de personas sobre la de precios, en los campos donde no hubiera precios nos aparecería un NULL.
 
   name   |  city   | favorite_color| price
:--------:|:-------:|:--------------:|----
  Juan    | Inca |     amarillo | 49.99 
 Maria    | Palma |     rojo | 59.99
  Pedro | Campanet | marron | NULL


En sintaxis sql, esto se escribiría:

```sql
SELECT f.name,
       p.price
FROM friends as f
LEFT JOIN prices as p
ON f.favorite_color = p.sweater_color
```

#### Ejercicio: Haz un left join de la tabla de alumnos a la de notas y selecciona las columnas name, year y nota

#### Ejercicio: Quien es el mejor alumno del año 2017

##### Ejercicio: Cual es la asignatura que tiene peor nota media de todas

##### Ejercicio: Que estudiantes tienen una nota media menor que 5. Agrupalo por años.

# Un caramelo: pandas

Ahora que ya sabemos realizar queries a bases de datos, con las que podemos filtrar y agregar, cómo podemos realizar un análisis más exhaustivo de los datos obtenidos?

Python permite connectarse a bases de datos de una forma sencilla y guardar los resultados en un formato que nos gusta a todos, los pandas.DataFrame()

Para poder conectarnos a nuestra base de datos, necesitaremos las librerias específicas, las más conocidas son:

* sqlite3: para pequeñas bases de datos
* sqlalchemy: se comunica con casi todo
* psycopg2: Postgres, redshift (aws)

Para ejecutar una query desde pandas, se deberá crear una conexión y pasar la query de forma:

```python
import pandas as pd
import sqlite3

conn = sqlite3.connect('bdd_notas.sqlite')
query = """SELECT * from asignaturas"""
df = pd.read_sql(query, conn)
```

De forma similar, podemos crear una tabla a partir de un dataframe de forma sencilla, por ejemplo desde un csv:

```python
import pandas as pd
import sqlite3

conn = sqlite3.connect('my_bdd.sqlite')
df = pd.read_csv("some_data.csv
df.to_sql('table_name', conn, if_exists='replace', index=False)
```

#### Ejercicio: Repite el último ejercicio y guarda los resultados en un dataframe de pandas


### Más info:

SQL:

http://www.w3schools.com/sql/default.asp

http://www.w3schools.com/sql/sql_quickref.asp

Relational databases and the normal forms:

http://en.wikipedia.org/wiki/Database_normalization#Normal_forms
    